# **1. Loading Data & Pre-process**

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

## **1.1 Data Executed**

In [ ]:
Data_exe = gc.open_by_key('1ewPVHJgoQI2fEOgTG3BHAEmgnSE1bCiDADnxr-xFmG0').worksheet ("Executed Contract")
Data_exe = pd.DataFrame (Data_exe.get_all_records())
Data_exe = Data_exe.replace(',','.',regex = True)

In [ ]:
Data_exe.head(2)

,Agreement Code,Agreement Name,Primary Owner,Created Date,Parent Agreement Code,Organization Unit,Execution Date,Total Duration,Draft Duration,Waiting For Approval Duration,Approved Duration,Waiting For Internal Signature Duration,Waiting For External Signature Duration,On Hold Duration,Cancelled Duration
0,DistributionAgreement_LGL_115,AMANDEMEN I ATAS PERJANJIAN PASOKAN NITROGEN CAIR,Joshua AGI,1/5/2022,DistributionAgreement_LGL_461,/ptsamator/sales dept/sales west region/west c...,3/15/2022,68.89,0.0,7.01,41.78,0.0,20.1,0,0
1,DistributionAgreement_LGL_117,ADDENDUM 1 - PENAMBAHAN PERALATAN,Elvira Amanda,1/12/2022,DistributionAgreement_LGL_927,/ptsamator/sales dept/sales east region/east java,3/8/2022,54.74,0.0,0.04,0.00,54.7,0.0,0,0


**Pre-process**

In [ ]:
Data_exe['Amendment'] = Data_exe['Agreement Code'].str.split("_").str[0]
Data_exe['Amendment'] = Data_exe['Amendment'].str.split("-").str[0]


mid = Data_exe['Amendment']
Data_exe.drop(labels=['Amendment'], axis=1,inplace = True)
Data_exe.insert(2, 'Amendment', mid)


In [ ]:
Data_exe.head(1)

,Agreement Code,Agreement Name,Amendment,Primary Owner,Created Date,Parent Agreement Code,Organization Unit,Execution Date,Total Duration,Draft Duration,Waiting For Approval Duration,Approved Duration,Waiting For Internal Signature Duration,Waiting For External Signature Duration,On Hold Duration,Cancelled Duration
0,DistributionAgreement_LGL_115,AMANDEMEN I ATAS PERJANJIAN PASOKAN NITROGEN CAIR,DistributionAgreement,Joshua AGI,1/5/2022,DistributionAgreement_LGL_461,/ptsamator/sales dept/sales west region/west c...,3/15/2022,68.89,0.0,7.01,41.78,0.0,20.1,0,0


In [ ]:
Data_exe['Created Month'] = pd.DatetimeIndex(Data_exe['Created Date']).month

def monthly (itera) :
  if itera['Created Month'] == 1 :
    return "January"
  elif itera ['Created Month'] == 2 :
    return "February"
  elif itera ['Created Month'] == 3 :
    return "March"
  elif itera ['Created Month'] == 4 :
    return "April"
  elif itera ['Created Month'] == 5 :
    return "May"
  elif itera ['Created Month'] == 6 :
    return "June"  
  elif itera ['Created Month'] == 7 :
    return "July"
  elif itera ['Created Month'] == 8 :
    return "August"
  elif itera ['Created Month'] == 9 :
    return "September"
  elif itera ['Created Month'] == 10 :
    return "October"
  elif itera ['Created Month'] == 11 :
    return "November"
  elif itera ['Created Month'] == 12:
    return "December"
  else :
    return "-"

Data_exe['Created Month'] = Data_exe.apply(monthly,axis =1)

mid2 = Data_exe['Created Month']
Data_exe.drop(labels=['Created Month'], axis=1,inplace = True)
Data_exe.insert(5, 'Created Month', mid2)

In [ ]:
def condition (iterasi) :
  if iterasi ['Parent Agreement Code'] == " " :
    return "Main Contract"
  else :
    return "Amendment"

Data_exe ['Amendment (Main Contract/Amendment)'] = Data_exe.apply(condition,axis =1 )

In [ ]:
Data_clas= Data_exe[['Amendment','Amendment (Main Contract/Amendment)']]
Data_exe['new'] = Data_clas.fillna(' ').sum(axis=1)

In [ ]:
def classif (iterate) :
  if iterate ['Amendment (Main Contract/Amendment)']== "Main Contract" :
    return iterate['Amendment']
  else :
    return iterate['new'] 

Data_exe['Classification'] = Data_exe.apply(classif, axis =1)
Data_exe.drop(['new'],axis=1,inplace=True)

## **1.2 Data Unxecuted**

In [ ]:
from datetime import datetime

today_unex = pd.Timestamp.now('Asia/Bangkok')
today_unex = pd.to_datetime(today_unex.strftime('%Y-%m-%d'))

In [ ]:
from gspread.models import Worksheet
Data_colum = pd.read_excel('/content/drive/MyDrive/Google Collaboration/Cycle Time Analysis/Cycle Time/220301 Cycle Time Analysis.xlsx',sheet_name = 'Raw Contract 2022')
Data_unex = pd.read_excel('/content/drive/MyDrive/Google Collaboration/Cycle Time Analysis/Cycle Time/Agreement.xlsx')
Data_unex_raw = pd.read_excel('/content/drive/MyDrive/Google Collaboration/Cycle Time Analysis/Cycle Time/Agreement.xlsx')

Columns = (Data_colum.columns) & (Data_unex.columns)
Data_unex = Data_unex[Columns]
Data_unex_raw = Data_unex_raw[Columns]

In [ ]:
Column = ['Created Date',
         'Effective Date',
          'Expiration Date'
        ]
          
Data_unex[Column] = Data_unex[Column].astype('datetime64[ns]')
Data_unex['Created Date'] = pd.to_datetime(Data_unex['Created Date'] , format='%Y-%m-%d')
Data_unex = Data_unex.loc[(Data_unex['Created Date'] >= '2022-01-01')
                     & (Data_unex['Created Date'] < today_unex )]

## **1.3 Remove Trial Agreement**

In [ ]:
list_str = "Coba","coba","Test","test", "tes"
for i in list_str:
  Data_unex = Data_unex[Data_unex['Agreement Title'].str.contains(i)==False]

# **2. Operation of Cycle Time**

In [ ]:
Data_ops = Data_unex.copy()

## 2.1 Execution Date

In [ ]:
Data_ex2 = Data_exe[['Agreement Code','Execution Date']]
Data_ex2['Agreement Code'] = Data_ex2['Agreement Code'].replace('-','_',regex=True) 
Data_ops['Agreement Code'] = Data_ops['Agreement Code'].replace('-','_',regex=True)

In [ ]:
Data_ops = pd.merge(Data_ops,Data_ex2,on = 'Agreement Code', how = 'left')

## 2.2 Agreement Code Check 
(Exclude NDA)

In [ ]:
Data_ops['Code_Check'] = Data_ops['Agreement Code'].str[:3]

def Agreement_code (i): 
  if i['Code_Check']=="Non" :
    return "Exclude"
  else :
    return "Include" 

Data_ops['Agreement Code Check'] = Data_ops.apply(Agreement_code,axis = 1)
Data_ops = Data_ops.drop(columns = 'Code_Check')
Data_ops

## 2.3 Superseded Check


In [ ]:
Data_ops['Status'].unique()

In [ ]:
def Superseded (it) :
  if it['Status'] == "Superseded" :
    return "Please delete if temination contract"
  else :
    return "Include"

Data_ops['Superseded Check'] = Data_ops.apply (Superseded,axis = 1)
Data_ops 


## 2.4 Creator Check 
(Exclude Kevin and Admins Entry)

In [ ]:
def Creator_Check (ite) :
  if ite['Created By'] == "Kevin Wijaya" or ite['Created By'] == "CLM Admin" :
    return "Exclude"
  else :
    return "Include"

Data_ops['Creator Check'] =  Data_ops.apply(Creator_Check,axis =1)


In [ ]:
Data_ops.drop(Data_ops.loc[Data_ops['Created By']=='Kevin Wijaya'].index,inplace = True)
Data_ops.drop(Data_ops.loc[Data_ops['Created By']=='CLM Admin'].index, inplace = True)

In [ ]:
Data_ops['Created By'].unique()

## 2.5 Contract Status Classification

In [ ]:
def Status_clas (iter) :
  if iter['Status'] == "Executed" or iter['Status'] == "Expired" or iter['Status'] == "Reviewed" or iter['Status'] == "Superseded" :
    return "Executed" 
  else :
    return "Unexecuted"

Data_ops['Contract Status Classification'] = Data_ops.apply(Status_clas, axis= 1)

## 2.6 Month

In [ ]:
Data_ops['Month'] =pd.DatetimeIndex(Data_ops['Created Date']).month

def month (itera) :
  if itera['Month'] == 1 :
    return "January"
  elif itera ['Month'] == 2 :
    return "February"
  elif itera ['Month'] == 3 :
    return "March"
  elif itera ['Month'] == 4 :
    return "April"
  elif itera ['Month'] == 5 :
    return "May"
  elif itera ['Month'] == 6 :
    return "June"  
  elif itera ['Month'] == 7 :
    return "July"
  elif itera ['Month'] == 8 :
    return "August"
  elif itera ['Month'] == 9 :
    return "September"
  elif itera ['Month'] == 10 :
    return "October"
  elif itera ['Month'] == 11 :
    return "November"
  elif itera ['Month'] == 12:
    return "December"
  else :
    return "-"


Data_ops['Month']  = Data_ops.apply(month,axis=1)

## 2.7 Ongoing Duration (Days)

In [ ]:
today = pd.Timestamp.now('Asia/Bangkok')
today = pd.to_datetime(today.strftime('%Y/%m/%d %H:%M:%S'))

Data_ops['Ongoing Duration (Days'] = today - Data_ops['Created Date']
Data_ops['Ongoing Duration (Days']= Data_ops['Ongoing Duration (Days'].dt.days.astype('int64')

## 2.8 Ongoing Duration (based on months)

**Create Lookup**

In [ ]:
import datetime
now = datetime.datetime.now()
result = [now.strftime("%B")]

for _ in range(1, 6):
    now = now.replace(day=1) - datetime.timedelta(days=1)
    result.append(now.strftime("%B"))
    
del result[0]

In [ ]:
row = []
for i in range(30,160,30):
  row.append(i)

Class = ['On Track',
        'On Track',
        '>60 Days',
        '>90 Days',
        '>120 Days']

Lookup = pd.DataFrame({'Month': result,'Days on Going':row,'Classification':Class})

**Operation**

In [ ]:
month_piv = Lookup[['Month','Classification']]

In [ ]:
Data_ops = pd.merge(Data_ops,month_piv, on = 'Month',how='left')
Data_ops = Data_ops.rename(columns ={'Classification':'On going Duration (Based on Month)'} )

## 2.9 Ongoing Duration (Based on Actual date)

In [ ]:
def actual1 (x) :
  if x['Ongoing Duration (Days'] > 30 :
    return Lookup['Classification'] [1:2]
  elif x['Ongoing Duration (Days'] > 60 :
    return Lookup['Classification'][:1]
  else : 
    return "On Track"

Data_ops['Ongoing Duration (Based on Actual date)'] = Data_ops.apply(actual1,axis=1)

In [ ]:
Data_ops.head(1)

# **3.Pivot Table of Cycle Time**

In [ ]:
Data_ops['Contractual Value'] = Data_ops['Contractual Value'].astype(str).str[:-3]
Data_ops['Contractual Value'] = Data_ops['Contractual Value'].replace(',','',regex = True)
Data_ops['Contractual Value'] = Data_ops['Contractual Value'].astype('int64')

## 3.1 Cycle Time Month

In [ ]:
Cycle_tim = pd.pivot_table(Data_ops,values = ['Agreement Title','Contractual Value'],
                        index = ['Month','On going Duration (Based on Month)'],
                        columns = 'Contract Status Classification',
                        margins = True, margins_name = 'Grand Total',
                        aggfunc = {'Agreement Title':'count', 'Contractual Value':sum}
                        )

In [ ]:
Cycle_tim

In [ ]:
Cycle_tim_ex = Cycle_tim.reset_index()
Cycle_tim_list = [Cycle_tim_ex.columns.to_list()] + Cycle_tim_ex.to_numpy().tolist()
Cycle_tim_list[0] = ['Month','On going Duration (Based on Month)', 'Executed','Unexecuted','Grand Total','Executed','Unexecuted','Grand Total' ]

## 3.2 Average Duration per Month

In [ ]:
Data_ops['Month'].unique()

In [ ]:
Ave_dur = Data_ops.pivot_table(values = 'Ongoing Duration (Days', index = 'Month',margins=True,margins_name='Average', aggfunc = 'mean' )
Ave_dur = Ave_dur.fillna(0)
Ave_dur

## 3.3 Average Cycle Time

In [ ]:
Ave_cy = pd.pivot_table(Data_exe, values = ['Total Duration','Agreement Code'],
                        index = ['Classification'],
                        columns = ['Created Month'],
                        margins = True, margins_name = 'Grand Total', fill_value = 0,
                        aggfunc ={'Total Duration': np.mean, 'Agreement Code': 'count'}
                         )
Ave_cy

In [ ]:
List_month = Data_exe['Created Month'].unique().tolist()

Ave = ['Classification']
Ave.extend(List_month)
Ave1 = ['Grand Total']
Ave.extend(Ave1)
Ave.extend (List_month)
Ave2 = ['Grand Total']
Ave.extend (Ave2)
Ave

In [ ]:
Ave_cy_ex = Ave_cy.reset_index()
Ave_cy_list = [Ave_cy_ex.columns.to_list()] + Ave_cy_ex.to_numpy().tolist()
Ave_cy_list[0] = Ave

## 3.4 Average Duration Status per PIC

In [ ]:
from numpy.ma.core import count

Data_obs = Data_ops.copy()

Data_obs.drop(Data_obs.loc[Data_obs['Status']=='Executed'].index,inplace = True)
Data_obs.drop(Data_obs.loc[Data_obs['Status']=='Expired'].index,inplace = True)

Data_obs['Status'].unique()
ave_duration = pd.pivot_table(Data_obs,values = ['Agreement Title','Ongoing Duration (Days'], 
                              index = ['Status','Created By'],
                              margins=True,margins_name='Average',fill_value=0,
                              aggfunc = {'Agreement Title':'count','Ongoing Duration (Days':np.mean})
ave_duration

## **3.5 Unexecuted**

In [ ]:
piv_unex = pd.pivot_table(Data_ops, values = 'Agreement Title',
                          index= ['Regions', 'Created By','Status'],
                          aggfunc='count' ,
                          margins=True, margins_name='Total')

In [ ]:
ave_duration

# **4. Export To Google Sheet**

## **4.1 Rename File**

In [ ]:
from datetime import date
now = date.today()
Now = now.strftime("%Y%m%d")
Now = Now[2:]
rename_file = "Cycle Time Analysis"+" "+"("+Now+")"

## **4.2 Export Dataframe**

**Raw Contract 2022**

In [ ]:
Data_unex_raw = Data_unex_raw.fillna('')

In [ ]:
Update_data= gc.open_by_key('1Z8NS4mdnN46r7KdJW4ZaqlNGdnb0WQENXwz_3VyjaDM').worksheet("Raw Contract 2022")
Update_data.clear()
Update_data.update('A1', [Data_unex_raw.columns.to_list()] + Data_unex_raw.to_numpy().tolist())

**Edit Contract 2022**

In [ ]:
convert = ['Created Date', 'Effective Date', 'Expiration Date']

Data_ops[convert] = Data_ops[convert].astype('str')
Data_ops = Data_ops.fillna('')

In [ ]:
Update_data= gc.open_by_key('1Z8NS4mdnN46r7KdJW4ZaqlNGdnb0WQENXwz_3VyjaDM').worksheet("Edit Contract 2022")
Update_data.clear()
Update_data.update('A1', [Data_ops.columns.to_list()] + Data_ops.to_numpy().tolist())

**Lookup**

In [ ]:
Update_data= gc.open_by_key('1Z8NS4mdnN46r7KdJW4ZaqlNGdnb0WQENXwz_3VyjaDM').worksheet("Lookup")
Update_data.clear()
Update_data.update('A1', [Lookup.columns.to_list()] + Lookup.to_numpy().tolist())

**Compyle Cycle**

In [ ]:
Update_data= gc.open_by_key('1Z8NS4mdnN46r7KdJW4ZaqlNGdnb0WQENXwz_3VyjaDM').worksheet("Compyle Cycle")
Update_data.clear()
Update_data.update('A1',[Data_exe.columns.to_list()] + Data_exe.to_numpy().tolist())

## **4.3 Export Pivottable**

In [ ]:
Ave_dur_ex             =   Ave_dur.reset_index()
ave_duration_ex        =   ave_duration.reset_index()
piv_unex_ex            =   piv_unex.reset_index()

In [ ]:
Update_data= gc.open_by_key('1Z8NS4mdnN46r7KdJW4ZaqlNGdnb0WQENXwz_3VyjaDM').worksheet("Pivot")
Update_data.clear()
Update_data.update('A1', rename_file)

Update_data.update('B3', [['Cycle Time Month']])
Update_data.update('B5', Cycle_tim_list)
Update_data.update('D4', [['Agreement Title']])
Update_data.update('G4', [['Contractual Value']])

Update_data.update('G13', [['Average Duration per Month']])
Update_data.update('G14', [Ave_dur_ex.columns.to_list()]+ Ave_dur_ex.to_numpy().tolist())

Update_data.update('L3',[['Average Cycle Time']])
Update_data.update('L5', Ave_cy_list)
Update_data.update('M4',[['Agreement Code']])
Update_data.update('Q4',[['Total Duration']])

Update_data.update('G21', [['Average Duration Status per PIC']])
Update_data.update('G22', [ave_duration_ex.columns.to_list()] + ave_duration_ex.to_numpy().tolist())

Update_data.update('B13', [['Pivot Unexecuted']])
Update_data.update('B14', [piv_unex_ex.columns.to_list()] + piv_unex_ex.to_numpy().tolist())